In [1]:
pip install pyTelegramBotAPI

     |████████████████████████████████| 147 kB 24.9 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.4.0-py3-none-any.whl size=128152 sha256=a6abab22a371fe7aa68f78f2bfc6f0d7cffc16d54c90584787becb7d6279d9fb
  Stored in directory: /root/.cache/pip/wheels/1e/6d/ff/435b12799b8212017f08dbbfdb7a1e5174d72d20fb7c0f4703
Successfully built pyTelegramBotAPI


In [ ]:
from bs4 import BeautifulSoup
import requests
import json
from urllib.parse import urlparse
import telebot;
from telebot import types
bot = telebot.TeleBot('5260450850:AAGkp80bHug3CzO0AVj41IEoVNDocOJLfGk');

In [ ]:
@bot.message_handler(commands=["start"])
def start(m, res=False):
        bot.send_message(m.chat.id,'Z')
        markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
        item1=types.KeyboardButton("Начать")
        item2=types.KeyboardButton("Информация")
        item3=types.KeyboardButton("Помощь")
        markup.add(item1)
        markup.add(item2)
        markup.add(item3)
        bot.send_message(m.chat.id, 'Нажми: \nНачать(чтобы воспользоваться программой)\nИнформация(чтобы узанть о нас)\nПомощь(чтобы узнать о боте)\nХуй(если тебе есть, чем поделиться)',  reply_markup=markup)

In [ ]:
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
  if message.text == "Начать":
    bot.send_message(message.from_user.id, "Введите команду /go")
  if message.text == "Помощь":
      bot.send_message(message.from_user.id, "Если у вас есть вопросы по боту или работе программы, то задать их можно здесь @miCherex")
  if message.text == "Информация":
      bot.send_message(message.from_user.id, "Это проект учащихся IT-класса лицея №1511 при МИФИ, созданный для защиты от фейковых новостей в сети.")
 

In [ ]:
@bot.message_handler(commands=["go"], content_types=['text'])
def go(message):
    global link;
    bot.send_message(message.from_user.id,'Введите ссылку на новость')
    link=message.text
    req = requests.get(link).text
    soup = BeautifulSoup(req, "lxml")
    head = soup.find('head')
    zagolovok = head.find('title').text
    textp = soup.find_all('div', attrs={'class': "article__text"})
    if (len(textp)) == 0:
        textp = soup.find_all('span', attrs={'class': 'intro'})
    if (len(textp)) == 0:
        text = soup.find_all('span', attrs={'class': 'intro'})
    if (len(textp)) == 0:
        textp = soup.find_all('p')
    data = soup.find_all('time')
    domain = urlparse(link).netloc
    if (int(domain.find("www.")) != -1):
      domain = domain[4:]
    if ((domain =='cnn.com' or domain =='money.cnn.com') and (len(data)==0)):
      

      data = soup.find_all('p', attrs={'class': 'update-time'})


    if len(data) == 0:
        data = soup.find_all('span', attrs={'class': 'article__info-date-modified'})
    if len(data) == 0:
        data = soup.find_all('div', attrs={'class': 'article__info-date-modified'})

    if len(data) == 0:
      data = soup.find_all('span', attrs={'class': 'DateLine__date___12trWy'})
    if len(data) == 0:
      data = soup.find_all('font', attrs={'style': 'vertical-align: inherit;'})
    if len(data) == 0:
      data = soup.find_all('div', attrs={'class': 'css-19m31ns'})
    if len(data) == 0:
      data = soup.find_all('p', attrs={'class': 'update-time'}) 
    if len(data) == 0:
      data = soup.find_all('span', attrs={'class': 'update-time'})
    if len(data) == 0:
      data = soup.find_all('div', attrs={'class': 'update-time'})#http://www.cnn.com/2017/10/09/politics/melania-trump-ivana-trump-first-lady/index.html
    if len(data)==0:
      data = soup.find_all('p', attrs={'class': 'update-time'})
      print(len(data))
    if (len(data) ==0 and domain=="activistpost.com"):
      data = soup.find_all('span', attrs={'class': 'entry-meta-date updated'}) # https://www.activistpost.com/2017/09/september-11-2001-questions-believe-official-story.html
    if len(data) == 0:
      data = soup.find_all('p', attrs={'class': 'entry-meta'})
    if len(data) == 0:
      data = soup.find_all('div', attrs={'class': "post_date"}) 
   
    data1=data[0].text
    # print(zagolovok)

  

    # for i in range(len(textp)):
    #     print(textp[i].text)

    # if len(data) != 0:
    #     for i in range(len(data)):
    #         print(data[i].text)
    # else:
    #     print("дата публикации на новстоном ресусре не указана")

    # print(data.text)

  

    url = link.split("//")[-1].split("/")[0].split('?')[0]
    show = "https://input.payapi.io/v1/api/fraud/domain/age/" + domain

    r = requests.get(show, verify = False)
    soupp = BeautifulSoup(r.text, "lxml")

    if r.status_code == 200:

        age = soupp.find('body').text
        str = int(age.find("Reg"))
        end = int(age.find("}"))
        # print(age[str:(end - 1)])

        data = r.text
        jsonToPython = json.loads(data)
        list = [zagolovok, data1, age[str:(end - 1)], domain]
        bot.send_message(message.from_user.id, text=list)
bot.polling()
  